In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib as plt
import re as ree

In [2]:
def vv4(row):
    row = str(row)
    X = [1,1,1]
    X[0] = row[0:4]
    X[1] = row[4:6]
    X[2] = row[6:8]
    return X[0]+'-'+X[1]+'-'+X[2]

def DriveYearMonthDay(d):
    d['jalaliDate'] = d['jalaliDate'].astype(str)
    d['Year'] = d['jalaliDate'].str[0:4]
    d['Month'] = d['jalaliDate'].str[4:6]
    d['Day'] = d['jalaliDate'].str[6:8]
    d['jalaliDate'] = d['jalaliDate'].astype(int)
    return d

In [3]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"

In [5]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"
n = path + '\Cleaned_Stocks_Holders_1399-07-21_From96.csv'
df = pd.read_csv(n)

df = df.drop(df.loc[df['Holder']=='شخص حقیقی'].index)
df = df.drop(df[(df['Trade']=='No')&((df['close_price']==10)|(df['close_price']==1000)|(df['close_price']==10000)|(df['close_price']==100000))].index)
df = df.drop(df[(df['symbol'] == 'وقوام')&(df['close_price'] == 1000)].index)
symbols = [ 'سپرده','هما','وهنر-پذیره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['symbol'].isin(symbols)].index)
df = df.drop(df[(df.symbol == 'اتکای')&(df.close_price == 1000)].index)

In [6]:
HolderData = df

In [8]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"
n1 = path + '\Stocks_Prices_1399-07-20' + '.csv'
df1 = pd.read_csv(n1)

df1['jalaliDate'] = df1['jalaliDate'].apply(vv)
df = df1
symbols = [ 'سپرده','هما','وهنر-پذيره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['name'].isin(symbols)].index)
df = df.drop(df[df.group_name== 'صندوق سرمایه گذاری قابل معامله'].index)
df = df.drop(df[(df.name == 'اتکای')&(df.close_price == 1000)].index)
df = df.drop_duplicates()
df = df.drop(df.loc[(df['volume'] == 0)].index).sort_values(by =['name','jalaliDate']).drop(columns = ['volume','quantity']).rename(columns = {'name':'symbol'})
df = DriveYearMonthDay(df)

In [9]:
PriceData = df

In [14]:
PriceData = PriceData[['jalaliDate', 'date', 'symbol','close_price']]
PriceData['date1'] = PriceData['date'].apply(vv4)
PriceData['date1'] = pd.to_datetime(PriceData['date1']) 
PriceData['week_of_year'] = PriceData['date1'].dt.week 
PriceData['year_of_year'] = PriceData['date1'].dt.year

In [16]:
def weekRet(g):
    g['weekRet'] = (g.close_price.iloc[-1] - g.close_price.iloc[0])/g.close_price.iloc[0] * 100
    return g

gg = PriceData.groupby(['symbol', 'year_of_year','week_of_year'])
d = gg.apply(weekRet)

In [40]:
weekdata = d.groupby(['symbol','year_of_year','week_of_year']).last().reset_index()

In [41]:
len(weekdata)

287321

In [17]:
n = path + '\Analyzed Data'+'\Factors.xlsx'
Factors = pd.read_excel(n)

In [42]:
weekdata = weekdata.merge(Factors,on = ['year_of_year','week_of_year']).sort_values(by =['symbol','date']).reset_index(drop=True).dropna()

In [140]:
n = path + '\RiskFree.xlsx'
df3 = pd.read_excel(n)
df3 = df3.rename(columns = {'Unnamed: 2' : 'Year'})
df3['YM'] = df3['YM'].astype(str)
df3['YM'] = df3['YM'] + '00'
df3['YM'] = df3['YM'].astype(int)
df4 = weekdata
df4['RiskFree'] = np.nan
df4['jalaliDate'] = df4['jalaliDate'].astype(int)
for i in df3.YM:
    df4.loc[df4.jalaliDate>= i,'RiskFree'] = df3.loc[df3['YM'] == i ].iloc[0,1]/12/52

In [144]:
weekdata = df4
weekdata['EMarketweekRet'] = weekdata['MarketweekRet'] - weekdata['RiskFree']
weekdata['EweekRet'] = weekdata['weekRet'] - weekdata['RiskFree']

In [148]:
wg = weekdata.groupby(['symbol'])
def FourFactor(g):
    try:
        y = 'EweekRet'
        x = ['EMarketweekRet','HML','SMB','Winner_Loser']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['4-Residual'] = g ['EweekRet'] - (model.params[0] + model.params[1]*g['EMarketweekRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB']  + model.params[4]*g['Winner_Loser']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

def ThreeFactor(g):
    try:
        y = 'EweekRet'
        x = ['EMarketweekRet','HML','SMB']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['3-Residual'] = g ['EweekRet'] - (model.params[0] + model.params[1]*g['EMarketweekRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB'] ) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g


re = wg.apply(FourFactor)
print('Fourfactor done')
re2 = wg.apply(ThreeFactor)
re['3-Residual'] = re2['3-Residual']
re = re.reset_index(drop=True)

انفورماتيك
ايران زمين
تعاون
خليبل
ريخته گري تبريز
وسبوشهر
وسيلام
وسگلستا
پترو گچساران
پترومرواريد
Fourfactor done
انفورماتيك
ايران زمين
تعاون
خليبل
ريخته گري تبريز
وسبوشهر
وسيلام
وسگلستا
پترو گچساران
پترومرواريد


In [149]:
residuals = re[re.jalaliDate>13960000]

In [111]:
len(set(residuals.symbol))
A = set(residuals.symbol)

676

In [121]:
C = set(HolderData.symbol)
len(set(HolderData.symbol))


647

In [155]:
def convert_ar_characters(input_str):

    mapping = {
        'ك': 'ک',
        'گ':'گ',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return _multiple_replace(mapping, input_str)


def _multiple_replace(mapping, text):
    pattern = "|".join(map(ree.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

col = 'symbol'
residuals[col] = residuals[col].apply(lambda x: convert_ar_characters(x))

In [156]:
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['4-Residual']))
HolderData['4-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)

fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['3-Residual']))
HolderData['3-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)

fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['weekRet']))
HolderData['weekRet'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)

In [157]:
WeekHolderData = HolderData.drop(HolderData[HolderData['4-Residual'].isnull()].index)

In [158]:

B = set(WeekHolderData.symbol)
len(set(WeekHolderData.symbol))

645

In [159]:
WeekHolderData

,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,shrout,close_price,Number_Change,Percent_Change,Condition,Trade,4-Residual,3-Residual,weekRet
1705,20170326,13960106,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,8.000000e+08,998.0,0.0,0.0,Open,Yes,-5.628725,-6.044962,-0.795229
1706,20170326,13960106,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,8.000000e+08,998.0,0.0,0.0,Open,Yes,-5.628725,-6.044962,-0.795229
1707,20170326,13960106,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وانصار,121014422.0,bank,15.12,8.000000e+08,998.0,0.0,0.0,Open,Yes,-5.628725,-6.044962,-0.795229
1708,20170326,13960106,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,8.000000e+08,998.0,0.0,0.0,Open,Yes,-5.628725,-6.044962,-0.795229
1709,20170326,13960106,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,322082470.0,private,40.26,8.000000e+08,998.0,0.0,0.0,Open,Yes,-5.628725,-6.044962,-0.795229
1720,20170329,13960109,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,8.000000e+08,1064.0,0.0,0.0,Open,Yes,3.389166,3.158753,5.242334
1721,20170329,13960109,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,8.000000e+08,1064.0,0.0,0.0,Open,Yes,3.389166,3.158753,5.242334
1722,20170329,13960109,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وانصار,121014422.0,bank,15.12,8.000000e+08,1064.0,0.0,0.0,Open,Yes,3.389166,3.158753,5.242334
1723,20170329,13960109,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,8.000000e+08,1064.0,0.0,0.0,Open,Yes,3.389166,3.158753,5.242334
1724,20170329,13960109,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,322082470.0,private,40.26,8.000000e+08,1064.0,0.0,0.0,Open,Yes,3.389166,3.158753,5.242334


In [160]:
df = WeekHolderData

In [161]:
Holders = list(set(df.Holder))
Holders.sort()
ids = list(range(len(Holders)))
mapingdict = dict(zip(Holders, ids))
df['Holder_id'] = df['Holder'].map(mapingdict)

symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(symbols, ids))
df['id'] = df['symbol'].map(mapingdict)

In [162]:
df['date1'] = df['date'].apply(vv4)
df['date1'] = pd.to_datetime(df['date1']) 
df['week_of_year'] = df['date1'].dt.week 
df['month_of_year'] = df['date1'].dt.month 
df['year_of_year'] = df['date1'].dt.year

In [163]:
df['MarketCap'] = df['close_price'] * df['shrout']

In [164]:
gdata = df.groupby(['id'])
g = gdata.get_group(2)

g.columns

Index(['date', 'jalaliDate', 'symbol', 'stock_id', 'group_name', 'group_id',
       'Holder', 'nshares', 'type', 'Percent', 'shrout', 'close_price',
       'Number_Change', 'Percent_Change', 'Condition', 'Trade', '4-Residual',
       '3-Residual', 'weekRet', 'Holder_id', 'id', 'date1', 'week_of_year',
       'month_of_year', 'year_of_year', 'MarketCap'],
      dtype='object')

In [165]:
data = pd.DataFrame()
gg = df.groupby(['id'])

def FCAPf(S_g,g):
    intersection = list(set.intersection(set(S_g.date), set(g.date)))
    g = g.loc[g.date.isin(intersection)]
    S_g = S_g.loc[S_g.date.isin(intersection)]
    a = g.merge(S_g , on = ['Holder_id','date'])
    if len(a) == 0: 
        return
    else:
        intersection = list(set.intersection(set(S_g.date), set(g.date)))
        g = g.loc[g.date.isin(intersection)]
        S_g = S_g.loc[S_g.date.isin(intersection)]
        a = g.merge(S_g , on = ['Holder_id','date','jalaliDate','week_of_year','month_of_year','year_of_year'])
        a['FCAPf'] = (a['nshares_x']*a['close_price_x'] + a['nshares_y']*a['close_price_y'])/(a['shrout_x']*a['close_price_x'] + a['shrout_y']*a['close_price_y'])
        a['SizeRatio'] = (a['MarketCap_x'])/(a['MarketCap_y'])

        f = a.groupby(['date','jalaliDate','id_x','id_y','week_of_year','month_of_year','year_of_year','group_name_x','group_name_y','weekRet_x','weekRet_y','SizeRatio','MarketCap_x','MarketCap_y','4-Residual_x','4-Residual_y','3-Residual_x','3-Residual_y'])['FCAPf'].sum().to_frame().reset_index()
        fc = f.groupby(['year_of_year','month_of_year'])[['4-Residual_x','4-Residual_y','3-Residual_x','3-Residual_y']].corr().reset_index()
        FourCor = fc.loc[fc.level_2 == '4-Residual_y'][['year_of_year','month_of_year','4-Residual_x']].rename(columns = {'4-Residual_x':'ρ_4'})
        ThreeCor = fc.loc[fc.level_2 == '3-Residual_y'][['year_of_year','month_of_year','3-Residual_x']].rename(columns = {'3-Residual_x':'ρ_3'})
        TimeId = zip(list(FourCor.year_of_year),list(FourCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(FourCor.ρ_4)))
        f['ρ_4'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        TimeId = zip(list(ThreeCor.year_of_year),list(ThreeCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(ThreeCor.ρ_3)))
        f['ρ_3'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)  
#         f['FCAPF'] = (f['FCAPf'] - f['FCAPf'].mean()).divide(f['FCAPf'].std())
        return f

for i in gg.groups.keys():
    g = gg.get_group(i)
    F_id = g.id.iloc[0]
    print(F_id)
    Next_df = df[df.id > F_id]
    S_gg = Next_df.groupby(['id'])
    data = data.append(S_gg.apply(FCAPf , g = g ))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [166]:
d = data.reset_index(drop = True).drop_duplicates(['id_x','id_y','month_of_year','year_of_year']).reset_index(drop = True)

In [167]:
symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(ids,symbols))
d['symbol_x'] = d['id_x'].map(mapingdict)
d['symbol_y'] = d['id_y'].map(mapingdict)
d

,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,weekRet_x,...,4-Residual_x,4-Residual_y,3-Residual_x,3-Residual_y,FCAPf,ρ_4,ρ_3,FCAPF,symbol_x,symbol_y
0,20170329,13960109,0,5,13,3,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,5.242334,...,3.389166,0.002425,3.158753,0.461321,0.021474,NaN,NaN,0.278095,آ س پ,آریان
1,20170409,13960120,0,5,14,4,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.928906,...,-1.573575,-11.719345,-1.529939,-11.909293,0.021429,-0.057728,-0.140464,0.248674,آ س پ,آریان
2,20170507,13960217,0,5,18,5,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.785855,...,-1.538400,-0.536257,-2.206891,0.499134,0.021408,-0.861590,-0.878724,0.235142,آ س پ,آریان
3,20170611,13960321,0,5,23,6,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.456816,...,-4.125419,-2.729581,-3.782695,-3.064560,0.021330,0.921933,0.952601,0.184917,آ س پ,آریان
4,20170709,13960418,0,5,27,7,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.107296,...,-3.459691,4.204767,-3.537203,4.481747,0.021310,-1.000000,-1.000000,0.172141,آ س پ,آریان
5,20170903,13960612,0,5,35,9,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.582278,...,-0.317473,-0.701879,-0.495424,-0.473587,0.021313,-0.289536,-0.033394,0.173739,آ س پ,آریان
6,20171008,13960716,0,5,40,10,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.258427,...,-1.009837,-1.789546,-0.724407,-2.208713,0.021224,-0.533325,-0.512306,0.116156,آ س پ,آریان
7,20171105,13960814,0,5,44,11,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,0.000000,...,-3.572110,0.064688,-3.343296,-0.123867,0.021260,-0.859162,-0.893016,0.139368,آ س پ,آریان
8,20171203,13960912,0,5,48,12,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.457143,...,-0.217892,-1.714724,-0.048347,-1.871698,0.021236,-0.752468,-0.803621,0.124163,آ س پ,آریان
9,20180114,13961024,0,5,2,1,2018,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-2.836879,...,-2.014544,-1.003925,-2.040725,-1.028398,0.021176,NaN,NaN,0.084898,آ س پ,آریان


In [ ]:
d['FCAPF'] = d['FCAPf'].transform('rank')
d['FCAPF'] = (d['FCAPF']-d['FCAPF'].mean())/d['FCAPF'].std()

In [168]:
d['month_of_year'] = d['month_of_year'].astype(str)
def add(row):
    if len(row)<2:
        row = '0'+ row
    return row

d['month_of_year'] = d['month_of_year'].apply(add )

In [169]:
d['year_of_year'] = d['year_of_year'].astype(str)

d['Year_Month'] = d['year_of_year'] + d['month_of_year']
d

,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,weekRet_x,...,4-Residual_y,3-Residual_x,3-Residual_y,FCAPf,ρ_4,ρ_3,FCAPF,symbol_x,symbol_y,Year_Month
0,20170329,13960109,0,5,13,03,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,5.242334,...,0.002425,3.158753,0.461321,0.021474,NaN,NaN,0.278095,آ س پ,آریان,201703
1,20170409,13960120,0,5,14,04,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.928906,...,-11.719345,-1.529939,-11.909293,0.021429,-0.057728,-0.140464,0.248674,آ س پ,آریان,201704
2,20170507,13960217,0,5,18,05,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.785855,...,-0.536257,-2.206891,0.499134,0.021408,-0.861590,-0.878724,0.235142,آ س پ,آریان,201705
3,20170611,13960321,0,5,23,06,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.456816,...,-2.729581,-3.782695,-3.064560,0.021330,0.921933,0.952601,0.184917,آ س پ,آریان,201706
4,20170709,13960418,0,5,27,07,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.107296,...,4.204767,-3.537203,4.481747,0.021310,-1.000000,-1.000000,0.172141,آ س پ,آریان,201707
5,20170903,13960612,0,5,35,09,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.582278,...,-0.701879,-0.495424,-0.473587,0.021313,-0.289536,-0.033394,0.173739,آ س پ,آریان,201709
6,20171008,13960716,0,5,40,10,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.258427,...,-1.789546,-0.724407,-2.208713,0.021224,-0.533325,-0.512306,0.116156,آ س پ,آریان,201710
7,20171105,13960814,0,5,44,11,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,0.000000,...,0.064688,-3.343296,-0.123867,0.021260,-0.859162,-0.893016,0.139368,آ س پ,آریان,201711
8,20171203,13960912,0,5,48,12,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.457143,...,-1.714724,-0.048347,-1.871698,0.021236,-0.752468,-0.803621,0.124163,آ س پ,آریان,201712
9,20180114,13961024,0,5,2,01,2018,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-2.836879,...,-1.003925,-2.040725,-1.028398,0.021176,NaN,NaN,0.084898,آ س پ,آریان,201801


In [170]:
days = list(set(d.Year_Month))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t'] = d['Year_Month'].map(mapingdict)

In [171]:
d

,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,weekRet_x,...,3-Residual_x,3-Residual_y,FCAPf,ρ_4,ρ_3,FCAPF,symbol_x,symbol_y,Year_Month,t
0,20170329,13960109,0,5,13,03,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,5.242334,...,3.158753,0.461321,0.021474,NaN,NaN,0.278095,آ س پ,آریان,201703,0
1,20170409,13960120,0,5,14,04,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.928906,...,-1.529939,-11.909293,0.021429,-0.057728,-0.140464,0.248674,آ س پ,آریان,201704,1
2,20170507,13960217,0,5,18,05,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.785855,...,-2.206891,0.499134,0.021408,-0.861590,-0.878724,0.235142,آ س پ,آریان,201705,2
3,20170611,13960321,0,5,23,06,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.456816,...,-3.782695,-3.064560,0.021330,0.921933,0.952601,0.184917,آ س پ,آریان,201706,3
4,20170709,13960418,0,5,27,07,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.107296,...,-3.537203,4.481747,0.021310,-1.000000,-1.000000,0.172141,آ س پ,آریان,201707,4
5,20170903,13960612,0,5,35,09,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.582278,...,-0.495424,-0.473587,0.021313,-0.289536,-0.033394,0.173739,آ س پ,آریان,201709,6
6,20171008,13960716,0,5,40,10,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.258427,...,-0.724407,-2.208713,0.021224,-0.533325,-0.512306,0.116156,آ س پ,آریان,201710,7
7,20171105,13960814,0,5,44,11,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,0.000000,...,-3.343296,-0.123867,0.021260,-0.859162,-0.893016,0.139368,آ س پ,آریان,201711,8
8,20171203,13960912,0,5,48,12,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.457143,...,-0.048347,-1.871698,0.021236,-0.752468,-0.803621,0.124163,آ س پ,آریان,201712,9
9,20180114,13961024,0,5,2,01,2018,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-2.836879,...,-2.040725,-1.028398,0.021176,NaN,NaN,0.084898,آ س پ,آریان,201801,10


In [172]:
d['id_x'] = d['id_x'].astype(str)
d['id_y'] = d['id_y'].astype(str)
d['id'] = d['symbol_x'] + d['symbol_y']
ids = list(set(d.id))
ids.sort()
id = list(range(len(ids)))
mapingdict = dict(zip(ids, id))
d['id'] = d['id'].map(mapingdict)

In [173]:
d.head()

,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,weekRet_x,...,3-Residual_y,FCAPf,ρ_4,ρ_3,FCAPF,symbol_x,symbol_y,Year_Month,t,id
0,20170329,13960109,0,5,13,03,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,5.242334,...,0.461321,0.021474,NaN,NaN,0.278095,آ س پ,آریان,201703,0,0
1,20170409,13960120,0,5,14,04,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.928906,...,-11.909293,0.021429,-0.057728,-0.140464,0.248674,آ س پ,آریان,201704,1,0
2,20170507,13960217,0,5,18,05,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.785855,...,0.499134,0.021408,-0.861590,-0.878724,0.235142,آ س پ,آریان,201705,2,0
3,20170611,13960321,0,5,23,06,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.456816,...,-3.064560,0.021330,0.921933,0.952601,0.184917,آ س پ,آریان,201706,3,0
4,20170709,13960418,0,5,27,07,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.107296,...,4.481747,0.021310,-1.000000,-1.000000,0.172141,آ س پ,آریان,201707,4,0


In [176]:
d['ρ3_f'] = d.groupby(['id'])['ρ_3'].shift(-1)
d['ρ4_f'] = d.groupby(['id'])['ρ_4'].shift(-1)
d['ρ3_f2'] = d.groupby(['id'])['ρ_3'].shift(-2)
d['ρ4_f2'] = d.groupby(['id'])['ρ_4'].shift(-2)


In [200]:
market = df.groupby(['date','symbol']).first().reset_index()[['jalaliDate','date','symbol','shrout','close_price']]
market['MarketCap'] = market['close_price'] * market['shrout']
gm = market.groupby(['date'])

market['Percentile_Rank'] = gm.MarketCap.rank(pct = True) 


In [202]:
t = d

In [201]:
market

,jalaliDate,date,symbol,shrout,close_price,MarketCap,Percentile_Rank
0,13960105,20170325,خفنر,1.000000e+08,2726.0,2.726000e+11,0.111111
1,13960105,20170325,سبهان,5.500000e+07,16121.0,8.866550e+11,0.222222
2,13960105,20170325,سدشت,1.180000e+09,2286.0,2.697480e+12,0.666667
3,13960105,20170325,سفانو,3.500000e+08,3482.0,1.218700e+12,0.555556
4,13960105,20170325,سمازن,8.220000e+08,4761.0,3.913542e+12,0.777778
5,13960105,20170325,غمهرا,3.500000e+07,31129.0,1.089515e+12,0.444444
6,13960105,20170325,فجام,8.000000e+07,11845.0,9.476000e+11,0.333333
7,13960105,20170325,پارسیان,3.000000e+09,1938.0,5.814000e+12,1.000000
8,13960105,20170325,کوثر,2.500000e+09,1600.0,4.000000e+12,0.888889
9,13960106,20170326,آ س پ,8.000000e+08,998.0,7.984000e+11,0.206687


In [205]:
fkey = zip(list(market.symbol),list(market.date))
mapingdict = dict(zip(fkey,market['Percentile_Rank']))
t['Percentile_Rank_x'] = t.set_index(['symbol_x','date']).index.map(mapingdict)
t['Percentile_Rank_y'] = t.set_index(['symbol_y','date']).index.map(mapingdict)

In [206]:
t['size1'] = 0
t.loc[t.MarketCap_x>t.MarketCap_y,'size1'] = t[t.MarketCap_x>t.MarketCap_y]['Percentile_Rank_x']
t.loc[t.MarketCap_x<t.MarketCap_y,'size1'] = t[t.MarketCap_x<t.MarketCap_y]['Percentile_Rank_y']
t['size2'] = 0
t.loc[t.MarketCap_x>t.MarketCap_y,'size2'] = t[t.MarketCap_x>t.MarketCap_y]['Percentile_Rank_y']
t.loc[t.MarketCap_x<t.MarketCap_y,'size2'] = t[t.MarketCap_x<t.MarketCap_y]['Percentile_Rank_x']


In [207]:
t['SameSize'] =  t['size2'] -t['size1']
t

,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,weekRet_x,...,id,size1,size2,ρ3_f,ρ4_f,ρ3_f2,ρ4_f2,Percentile_Rank_x,Percentile_Rank_y,SameSize
0,20170329,13960109,0,5,13,03,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,5.242334,...,0,0.931122,0.278061,-0.140464,-0.057728,-0.878724,-0.861590,0.278061,0.931122,-0.653061
1,20170409,13960120,0,5,14,04,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.928906,...,0,0.936740,0.257908,-0.878724,-0.861590,0.952601,0.921933,0.257908,0.936740,-0.678832
2,20170507,13960217,0,5,18,05,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.785855,...,0,0.937198,0.214976,0.952601,0.921933,-1.000000,-1.000000,0.214976,0.937198,-0.722222
3,20170611,13960321,0,5,23,06,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.456816,...,0,0.938503,0.211230,-1.000000,-1.000000,-0.033394,-0.289536,0.211230,0.938503,-0.727273
4,20170709,13960418,0,5,27,07,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.107296,...,0,0.955614,0.234987,-0.033394,-0.289536,-0.512306,-0.533325,0.234987,0.955614,-0.720627
5,20170903,13960612,0,5,35,09,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.582278,...,0,0.936118,0.223587,-0.512306,-0.533325,-0.893016,-0.859162,0.223587,0.936118,-0.712531
6,20171008,13960716,0,5,40,10,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-3.258427,...,0,0.938389,0.210900,-0.893016,-0.859162,-0.803621,-0.752468,0.210900,0.938389,-0.727488
7,20171105,13960814,0,5,44,11,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,0.000000,...,0,0.930693,0.205446,-0.803621,-0.752468,NaN,NaN,0.205446,0.930693,-0.725248
8,20171203,13960912,0,5,48,12,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-0.457143,...,0,0.937349,0.207229,NaN,NaN,NaN,NaN,0.207229,0.937349,-0.730120
9,20180114,13961024,0,5,2,01,2018,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-2.836879,...,0,0.931280,0.196682,NaN,NaN,NaN,NaN,0.196682,0.931280,-0.734597


In [209]:
t['sgroup'] = 0
t.loc[t.group_name_x == t.group_name_y,'sgroup'] = 1

In [211]:
n3 = path + '\\NormalzedFCAP2.csv'
t.to_csv(n3,index = False)

In [212]:
path

'H:\\Economics\\Finance(Prof.Heidari-Aghajanzadeh)\\Data'

In [ ]:
d[d.id == 4006]

In [210]:
t[t.duplicated(['t','id'])]

,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,weekRet_x,...,size1,size2,ρ3_f,ρ4_f,ρ3_f2,ρ4_f2,Percentile_Rank_x,Percentile_Rank_y,SameSize,sgroup


In [6]:
n3 = path + '\\NormalzedFCAP2.csv'
df = pd.read_csv(n3)

In [41]:
df['FCAPF'] = df['FCAPf'].transform('rank')
df['FCAPF'] = (df['FCAPF']-df['FCAPF'].mean())/df['FCAPF'].std()

In [37]:
n4 = path + '\\NormalzedFCAP2.1.csv'
df.to_csv(n4,index = False)

In [42]:
df[['FCAPF','FCAPf']]

,FCAPF,FCAPf
0,-0.954748,0.021474
1,-0.956716,0.021429
2,-0.957548,0.021408
3,-0.961227,0.021330
4,-0.961974,0.021310
5,-0.961851,0.021313
6,-0.966202,0.021224
7,-0.964424,0.021260
8,-0.965540,0.021236
9,-0.968434,0.021176


In [43]:
df[['FCAPF','FCAPf']]

,FCAPF,FCAPf
0,-0.954748,0.021474
1,-0.956716,0.021429
2,-0.957548,0.021408
3,-0.961227,0.021330
4,-0.961974,0.021310
5,-0.961851,0.021313
6,-0.966202,0.021224
7,-0.964424,0.021260
8,-0.965540,0.021236
9,-0.968434,0.021176
